* By: Rino Hilman
* Email: rinohilman@yahoo.com
* Reference: Detection of False Investment Strategies using Unsupervised Learning Methods by Marcos Lopez de Prado and Michael J. Lewis 




# Optimal Number of Clusters (ONC)

* Warning : written for python 3


## Introduction

Optimal Number of Clusters algorithm has a purpose to detect optimal number of K-Means clusters using feature correlation matrix and silhouette scores.
This implementation is based on 'Detection of False Investment Strategies using Unsupervised Learning Methods' https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3167017

The result of this algorithm is a tupple that contains:

    Correlation Matrix
    Optimized Clusters
    Silhouette Scores

Correlation Matrix show the matrix that are sorted by their relevance. Optimized Clustres show the optimal number of clustres and each of the culsters' contents.


## Purpose

This notebook shows the modified K-Means clustering or known as ONC algorithm preceded by a random correlation matrix generator.
Both of the algorithms are shown below.

## Exercises

The exercises contain the working codes that represent the ONC algorithm and the random correlation matrix generator. All codes can be executed and will work accordingly. The first step is to import all of the necessary packages.

In [386]:
import numpy as np
import pandas as pd 
from scipy.linalg import block_diag 
from sklearn.utils import check_random_state 

## Random Correlation Block - Matrices Algorithm

A function getCovSub has a purpose to find the Sub Covariance Matrix

In [387]:
def getCovSub(nObs,nCols,sigma,random_state=None): 
    # Sub covariance matrix 
    rng = check_random_state(random_state) 
    if nCols==1:return np.ones((1,1)) 
    ar0=rng.normal(size=(nObs,1))     
    ar0=np.repeat(ar0,nCols,axis=1)     
    ar0+=rng.normal(scale=sigma,size=ar0.shape)     
    ar0=np.cov(ar0,rowvar=False)     
    return ar0

The function getRndBlockCov generates a random covariance matrix with a given number of blocks

In [388]:
def getRndBlockCov(nCols,nBlocks,minBlockSize=1,sigma=1.,random_state=None):
    # Generate a random covariance matrix with a given number of blocks     
    rng = check_random_state(random_state)     
    parts=rng.choice(range(1,nCols-(minBlockSize-1)*nBlocks),nBlocks-1,replace=False)     
    parts.sort()     
    parts=np.append(parts,nCols-(minBlockSize-1)*nBlocks)     
    parts=np.append(parts[0],np.diff( parts )) - 1 + minBlockSize     
    cov=None     
    for nCols_ in parts:         
        cov_=getCovSub(int(max(nCols_*(nCols_+1)/2.,100)),nCols_,sigma,random_state=rng)         
        if cov is None:cov=cov_.copy()         
        else:cov=block_diag(cov,cov_)     
    return cov 

The function cov2corr transforms covariance into correlation matrix

In [389]:
def cov2corr(cov):     
    # Derive the correlation matrix from a covariance matrix     
    std=np.sqrt(np.diag(cov))     
    corr=cov/np.outer(std,std)     
    corr[corr<-1],corr[corr>1]=-1,1 # numerical error     
    return corr 

The function randomBlockCorr forms the random block correlation 

In [390]:
def randomBlockCorr(nCols,nBlocks,random_state=None,minBlockSize=1):     
    # Form block correlation    
    rng = check_random_state(random_state)     
    cov0=getRndBlockCov(nCols,nBlocks,minBlockSize=minBlockSize,\
                        sigma=.5,random_state=rng) # perfect block corr     
    cov1=getRndBlockCov(nCols,1,minBlockSize=minBlockSize,\
                        sigma=1.,random_state=rng) # add noise     
    cov0+=cov1     
    corr0=cov2corr(cov0)     
    corr0=pd.DataFrame(corr0)     
    return corr0

## The Base Clustering Algorithm

The code brings in the base clustering that results in the correlation matrix, the clusters, and the Silhouette scores. The purpose of this step is to perform a first-pass estimate of E[𝐾]. First, we transform the correlation matrix into a distance matrix. On this distance matrix, we apply the K-means algorithm on alternative target number of clusters. For each target number of clusters, we perform a stochastic optimization, repeating the clustering operation n_init times. Among all the clustering alternatives, we choose the solution that achieves the highest quality score, defined as the t-value of the silhouette scores. 

In [391]:

#------------------------------------------------------------------------------ 
def clusterKMeansBase(corr0,maxNumClusters=10,n_init=10):     
    kmeans = 0
    from sklearn.cluster import KMeans     
    from sklearn.metrics import silhouette_samples     
    dist,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series() # distance matrix     
    for init in range(n_init):         
        for i in range(2,maxNumClusters+1): # find optimal num clusters             
            kmeans_=KMeans(n_clusters=i,n_jobs=1,n_init=1)             
            kmeans_=kmeans_.fit(dist)             
            silh_=silhouette_samples(dist,kmeans_.labels_)             
            stat=(silh_.mean()/silh_.std(),silh.mean()/silh.std())             
            if np.isnan(stat[1]) or stat[0]>stat[1]:                 
                silh,kmeans=silh_,kmeans_     
    n_clusters = len( np.unique( kmeans.labels_ ) )     
    newIdx=np.argsort(kmeans.labels_)     
    corr1=corr0.iloc[newIdx] # reorder rows     
    corr1=corr1.iloc[:,newIdx] # reorder columns     
    clstrs={i:corr0.columns[np.where(kmeans.labels_==i)[0] ].tolist() for\
            i in np.unique(kmeans.labels_) } # cluster members     
    silh=pd.Series(silh,index=dist.index)     
    return corr1,clstrs,silh


## The Top-Level Clustering Algorithm

The code performs a second pass-estimate of E[K], the number of clustered trials. We evaluate the quality score for each cluster within the first-pass solution. Those clusters with quality greater or equal than average remain unchanged. We re-run the base clustering on clusters with below-average quality. The outputs of these re-runs are preserved only if their cluster quality improves. 
 

In [392]:
def makeNewOutputs(corr0,clstrs,clstrs2):     
    from sklearn.metrics import silhouette_samples     
    clstrsNew,newIdx={},[]     
    for i in clstrs.keys():         
        clstrsNew[len(clstrsNew.keys())]=list(clstrs[i])     
    for i in clstrs2.keys():         
        clstrsNew[len(clstrsNew.keys())]=list(clstrs2[i])     
    map(newIdx.extend, clstrsNew.values())     
    corrNew=corr0.loc[newIdx,newIdx] 
    dist=((1-corr0.fillna(0))/2.)**.5     
    kmeans_labels=np.zeros(len(dist.columns))     
    for i in clstrsNew.keys():         
        idxs=[dist.index.get_loc(k) for k in clstrsNew[i]]         
        kmeans_labels[idxs]=i     
    silhNew=pd.Series(silhouette_samples(dist,kmeans_labels),index=dist.index)     
    return corrNew,clstrsNew,silhNew 
#------------------------------------------------------------------------------ 

def clusterKMeansTop(corr0,maxNumClusters=10,n_init=10):     
    corr1,clstrs,silh=clusterKMeansBase(corr0,maxNumClusters=corr0.shape[1]-1,n_init=n_init)     
    clusterTstats={i:np.mean(silh[clstrs[i]])/np.std(silh[clstrs[i]]) for i in clstrs.keys()}     
    tStatMean=np.mean(list(clusterTstats.values()))     
    redoClusters=[i for i in clusterTstats.keys() if clusterTstats[i]<tStatMean]     
    if len(redoClusters)<=2:         
        return corr1,clstrs,silh     
    else:         
        keysRedo=[];map(keysRedo.extend,[clstrs[i] for i in redoClusters])         
        corrTmp=corr0.loc[keysRedo,keysRedo]         
        meanRedoTstat=np.mean([clusterTstats[i] for i in redoClusters])         
        corr2,clstrs2,silh2=clusterKMeansTop(corrTmp,\
            maxNumClusters=corrTmp.shape[1]-1,n_init=n_init)         
        # Make new outputs, if necessary         
        corrNew,clstrsNew,silhNew=makeNewOutputs(corr0,\
            {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters},clstrs2)        
        newTstatMean=np.mean([np.mean(silhNew[clstrsNew[i]])/np.std(silhNew[clstrsNew[i]])\
            for i in clstrsNew.keys()])         
        if newTstatMean<=meanRedoTstat:             
            return corr1,clstrs,silh         
        else:             
            return corrNew,clstrsNew,silhNew 

## Execution

Set predetermined variables for the algorithm. These values are generated randomly according to necessities.

In [400]:
import random
nBlocks1 = random.randint(1,10)
nCols1= random.randint(nBlocks1,10)

Execute the randomBlockCorr function to create a random correlation matrix.

In [401]:
#randomBlockCorr(nCols,nBlocks,random_state=None,minBlockSize=1)
corr0A=randomBlockCorr(nCols1,nBlocks1,random_state=None,minBlockSize=1)

In [402]:
corr0A.head()

,0,1,2,3,4,5,6,7,8
0,1.000000,0.299803,0.367218,0.353776,0.320724,0.333638,0.287385,0.248304,0.295062
1,0.299803,1.000000,0.580793,0.313102,0.317575,0.235510,0.264361,0.305430,0.315029
2,0.367218,0.580793,1.000000,0.282453,0.270105,0.282295,0.261318,0.281958,0.317544
3,0.353776,0.313102,0.282453,1.000000,0.650832,0.603599,0.629012,0.601470,0.581284
4,0.320724,0.317575,0.270105,0.650832,1.000000,0.598514,0.610589,0.592051,0.596937


Execute the Top-Level Clustering Algorithm that yields in: 

* Correlation Matrix
* Optimized Clusters
* Silhouette Scores

In [403]:
#clusterKMeansTop(corr0,maxNumClusters=10,n_init=10)
clusterKMeansTop(corr0A,maxNumClusters=10,n_init=10)

(          3         4         5         6         7         8         0  \
 3  1.000000  0.650832  0.603599  0.629012  0.601470  0.581284  0.353776   
 4  0.650832  1.000000  0.598514  0.610589  0.592051  0.596937  0.320724   
 5  0.603599  0.598514  1.000000  0.570831  0.631035  0.565898  0.333638   
 6  0.629012  0.610589  0.570831  1.000000  0.635946  0.582609  0.287385   
 7  0.601470  0.592051  0.631035  0.635946  1.000000  0.651579  0.248304   
 8  0.581284  0.596937  0.565898  0.582609  0.651579  1.000000  0.295062   
 0  0.353776  0.320724  0.333638  0.287385  0.248304  0.295062  1.000000   
 1  0.313102  0.317575  0.235510  0.264361  0.305430  0.315029  0.299803   
 2  0.282453  0.270105  0.282295  0.261318  0.281958  0.317544  0.367218   
 
           1         2  
 3  0.313102  0.282453  
 4  0.317575  0.270105  
 5  0.235510  0.282295  
 6  0.264361  0.261318  
 7  0.305430  0.281958  
 8  0.315029  0.317544  
 0  0.299803  0.367218  
 1  1.000000  0.580793  
 2  0.580793 

The result of ONC can be seen above.

## Consistency checking with the package

We will check the consistency between the above steps on ONC with the mlfinlab ONC package that is known as get_onc_clusters.

In [404]:
from mlfinlab.clustering.onc import get_onc_clusters # import the ONC function

In [405]:
data2 = get_onc_clusters(corr0A,10)

In [406]:
data2

(          3         4         5         6         7         8         0  \
 3  1.000000  0.650832  0.603599  0.629012  0.601470  0.581284  0.353776   
 4  0.650832  1.000000  0.598514  0.610589  0.592051  0.596937  0.320724   
 5  0.603599  0.598514  1.000000  0.570831  0.631035  0.565898  0.333638   
 6  0.629012  0.610589  0.570831  1.000000  0.635946  0.582609  0.287385   
 7  0.601470  0.592051  0.631035  0.635946  1.000000  0.651579  0.248304   
 8  0.581284  0.596937  0.565898  0.582609  0.651579  1.000000  0.295062   
 0  0.353776  0.320724  0.333638  0.287385  0.248304  0.295062  1.000000   
 1  0.313102  0.317575  0.235510  0.264361  0.305430  0.315029  0.299803   
 2  0.282453  0.270105  0.282295  0.261318  0.281958  0.317544  0.367218   
 
           1         2  
 3  0.313102  0.282453  
 4  0.317575  0.270105  
 5  0.235510  0.282295  
 6  0.264361  0.261318  
 7  0.305430  0.281958  
 8  0.315029  0.317544  
 0  0.299803  0.367218  
 1  1.000000  0.580793  
 2  0.580793 

The step by step results in the exercises have the same consistency as the ONC mlfinlab package results.